# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatinating

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [1]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# Read in our outcomes data
# Use the parse_dates argument to parse DateTime and Date of Birth
outcomes = pd.read_csv('data/Austin_Animal_Center_Outcomes_053122.csv', 
                       parse_dates=['DateTime', 'Date of Birth'])

In [5]:
# Check it out
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [6]:
# Check the info
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139957 entries, 0 to 139956
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         139957 non-null  object        
 1   Name              98510 non-null   object        
 2   DateTime          139957 non-null  datetime64[ns]
 3   MonthYear         139957 non-null  object        
 4   Date of Birth     139957 non-null  datetime64[ns]
 5   Outcome Type      139934 non-null  object        
 6   Outcome Subtype   64116 non-null   object        
 7   Animal Type       139957 non-null  object        
 8   Sex upon Outcome  139956 non-null  object        
 9   Age upon Outcome  139936 non-null  object        
 10  Breed             139957 non-null  object        
 11  Color             139957 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.8+ MB


In [7]:
# Let's create our Age in Days column
outcomes['Calculated Age in Days'] = pd.to_datetime(outcomes['DateTime'].dt.date) - outcomes['Date of Birth']

In [8]:
# Grab just the integer here...
outcomes['Calculated Age in Days'] = outcomes['Calculated Age in Days'].dt.days

In [9]:
# Sanity check
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [12]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby(by='Animal Type').count()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
Animal Type,,,,,,,,,,,,
Bird,664,147,664,664,664,664,384,664,664,664,664,664
Cat,53092,31197,53092,53092,53092,53088,32388,53092,53091,53092,53092,53092
Dog,78736,66044,78736,78736,78736,78722,25316,78735,78731,78736,78736,78736
Livestock,25,3,25,25,25,25,19,25,25,25,25,25
Other,7440,1119,7440,7440,7440,7435,6009,7440,7425,7440,7440,7440


Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is `groups`.

In [14]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value
outcomes.groupby('Animal Type').groups.keys()

dict_keys(['Bird', 'Cat', 'Dog', 'Livestock', 'Other'])

In [15]:
# Same goes for multi-index groupbys
outcomes.groupby(['Animal Type', 'Sex upon Outcome']).groups.keys()

dict_keys([('Cat', 'Neutered Male'), ('Dog', 'Neutered Male'), ('Other', 'Unknown'), ('Cat', 'Intact Male'), ('Dog', 'Spayed Female'), ('Cat', 'Intact Female'), ('Dog', 'Intact Female'), ('Cat', 'Spayed Female'), ('Dog', 'Intact Male'), ('Cat', 'Unknown'), ('Other', 'Spayed Female'), ('Other', 'Intact Male'), ('Bird', 'Intact Male'), ('Dog', 'Unknown'), ('Other', 'Intact Female'), ('Other', 'Neutered Male'), ('Bird', 'Unknown'), ('Bird', 'Intact Female'), ('Livestock', 'Intact Female'), ('Livestock', 'Unknown'), ('Livestock', 'Neutered Male'), ('Livestock', 'Intact Male'), ('Dog', nan)])

In [ ]:
# .groups outputs a dictionary, so we can access the group names using keys()


In [17]:
# We can then get a specific group, such as cats that were adopted
#can think of this as a .loc statemeant too
outcomes.groupby(['Animal Type', 'Sex upon Outcome']).get_group(('Bird', 'Intact Female'))

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
2270,A827060,*Stevie Nicks,2020-12-30 10:20:00,Dec 2020,2019-12-11,Adoption,Foster,Bird,Intact Female,1 year,Parakeet,Blue/White,385
4757,A835507,*835507,2021-06-03 12:57:00,Jun 2021,2020-05-28,Transfer,Partner,Bird,Intact Female,1 year,Chicken,Black/White,371
6582,A690593,NaN,2014-10-29 11:56:00,Oct 2014,2013-10-23,Transfer,Partner,Bird,Intact Female,1 year,Chicken Mix,Brown/White,371
10153,A821655,NaN,2020-08-16 14:55:00,Aug 2020,2019-08-16,Euthanasia,Suffering,Bird,Intact Female,1 year,Chicken,Red/Brown,366
10395,A802697,Ocean,2019-08-21 19:00:00,Aug 2019,2018-08-21,Adoption,NaN,Bird,Intact Female,1 year,Budgerigar,Blue,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136888,A852347,NaN,2022-03-02 12:15:00,Mar 2022,2020-03-01,Adoption,NaN,Bird,Intact Female,2 years,Duck,Tricolor/White,731
136889,A852344,NaN,2022-03-02 12:12:00,Mar 2022,2020-03-01,Adoption,NaN,Bird,Intact Female,2 years,Duck,Tricolor/White,731
137986,A854498,NaN,2022-04-07 17:00:00,Apr 2022,2020-04-04,Return to Owner,NaN,Bird,Intact Female,2 years,Chicken,Buff/Brown,733
137988,A854499,NaN,2022-04-07 17:00:00,Apr 2022,2020-04-04,Return to Owner,NaN,Bird,Intact Female,2 years,Chicken,Buff/Brown,733


## Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [18]:
# Try using count
outcomes.groupby('Animal Type').count()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
Animal Type,,,,,,,,,,,,
Bird,664,147,664,664,664,664,384,664,664,664,664,664
Cat,53092,31197,53092,53092,53092,53088,32388,53092,53091,53092,53092,53092
Dog,78736,66044,78736,78736,78736,78722,25316,78735,78731,78736,78736,78736
Livestock,25,3,25,25,25,25,19,25,25,25,25,25
Other,7440,1119,7440,7440,7440,7435,6009,7440,7425,7440,7440,7440


In [19]:
outcomes.groupby('Animal Type').mean()

,Calculated Age in Days
Animal Type,
Bird,538.016566
Cat,532.792831
Dog,1000.154567
Livestock,503.800000
Other,483.761290


In [27]:
outcomes.groupby('Animal Type')['Calculated Age in Days'].agg(['mean', 'median'])

,mean,median
Animal Type,,
Bird,538.016566,373.5
Cat,532.792831,111.0
Dog,1000.154567,730.0
Livestock,503.800000,383.0
Other,483.761290,366.0


## Exercise

Use `.groupby()` to find the most recent birth date of each (main) animal type.


In [33]:
# Your code here
outcomes.groupby('Animal Type')['Date of Birth'].max()

Animal Type
Bird        2022-03-24
Cat         2022-05-28
Dog         2022-05-05
Livestock   2020-05-28
Other       2022-04-06
Name: Date of Birth, dtype: datetime64[ns]

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [34]:
# Group by Outcome Type and Sex upon Outcome
outcomes.groupby(['Outcome Type', 'Sex upon Outcome']).mean()

Calculated Age in Days
Outcome Type    Sex upon Outcome                        
Adoption        Intact Female                 420.980477
                Intact Male                   475.007550
                Neutered Male                 650.555269
                Spayed Female                 646.295841
                Unknown                       386.297980
Died            Intact Female                 348.707831
                Intact Male                   301.036145
                Neutered Male                1842.490385
                Spayed Female                2091.836735
                Unknown                       312.623626
Disposal        Intact Female                 416.108108
                Intact Male                   725.140625
                Neutered Male                1987.777778
                Spayed Female                2470.166667
                Unknown                       449.214700
Euthanasia      Intact Female                1125.071558
                Intact Male                   917.067214
                Neutered Male                2235.501742
                Spayed Female                2317.935185
                Unknown                       506.181462
Missing         Intact Female                 266.800000
                Intact Male                   340.846154
                Neutered Male                1188.285714
                Spayed Female                1262.733333
                Unknown                       169.250000
Relocate        Intact Female                 732.000000
                Neutered Male                1105.000000
                Spayed Female                 495.000000
                Unknown                       588.761905
Return to Owner Intact Female                1064.451399
                Intact Male                  1106.558726
                Neutered Male                1636.608068
                Spayed Female                1753.323775
                Unknown                       769.424731
Rto-Adopt       Intact Female                1455.724138
                Intact Male                  1484.439024
                Neutered Male                1249.344196
                Spayed Female                1241.921450
                Unknown                      1590.000000
Transfer        Intact Female                 407.642697
                Intact Male                   351.665780
                Neutered Male                1124.760147
                Spayed Female                1097.225019
                Unknown                       171.231835

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [35]:
# Check it out!
outcomes.pivot_table(index='Outcome Type', columns='Sex upon Outcome', aggfunc='mean')

Calculated Age in Days                             \
Sex upon Outcome          Intact Female  Intact Male Neutered Male   
Outcome Type                                                         
Adoption                     420.980477   475.007550    650.555269   
Died                         348.707831   301.036145   1842.490385   
Disposal                     416.108108   725.140625   1987.777778   
Euthanasia                  1125.071558   917.067214   2235.501742   
Missing                      266.800000   340.846154   1188.285714   
Relocate                     732.000000          NaN   1105.000000   
Return to Owner             1064.451399  1106.558726   1636.608068   
Rto-Adopt                   1455.724138  1484.439024   1249.344196   
Transfer                     407.642697   351.665780   1124.760147   

                                             
Sex upon Outcome Spayed Female      Unknown  
Outcome Type                                 
Adoption            646.295841   386.297980  
Died               2091.836735   312.623626  
Disposal           2470.166667   449.214700  
Euthanasia         2317.935185   506.181462  
Missing            1262.733333   169.250000  
Relocate            495.000000   588.761905  
Return to Owner    1753.323775   769.424731  
Rto-Adopt          1241.921450  1590.000000  
Transfer           1097.225019   171.231835

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [36]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [37]:
toy2

,age,MP
0,63,100
1,33,200


In [38]:
# We can't just join these as they are, since we haven't specified our suffixes

toy1.join(toy2)

ValueError: columns overlap but no suffix specified: Index(['age'], dtype='object')

In [39]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [40]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


In [41]:
toy1.drop('age', axis=1).join(toy2)

,HP,age,MP
0,142,63,100
1,47,33,200


## `.merge()`

Or we could use `.merge()`:

In [42]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [43]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [44]:
states = pd.read_csv('data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [45]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [46]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,NaN,NaN,NaN,OH,buckeye,Columbus
6,NaN,NaN,NaN,OR,beaver,Salem


## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [47]:
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

,cuisine,genre
0,Greek,horror
1,Indian,scifi
2,American,fantasy
3,Thai,tech
4,Indian,documentary


In [48]:
ds_full = pd.concat([ds_chars, prefs])
ds_full

,name,HP,home_state,cuisine,genre
0,greg,200.0,WA,NaN,NaN
1,miles,200.0,WA,NaN,NaN
2,alan,170.0,TX,NaN,NaN
3,alison,300.0,DC,NaN,NaN
4,rachel,200.0,TX,NaN,NaN
0,NaN,NaN,NaN,Greek,horror
1,NaN,NaN,NaN,Indian,scifi
2,NaN,NaN,NaN,American,fantasy
3,NaN,NaN,NaN,Thai,tech
4,NaN,NaN,NaN,Indian,documentary


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [49]:
ds_full = pd.concat([ds_chars, prefs], axis=1)
ds_full

,name,HP,home_state,cuisine,genre
0,greg,200,WA,Greek,horror
1,miles,200,WA,Indian,scifi
2,alan,170,TX,American,fantasy
3,alison,300,DC,Thai,tech
4,rachel,200,TX,Indian,documentary


## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [50]:
# Peek at the outcomes data we already had in here
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6


In [53]:
# Read in the intakes data
# We can parse our dates again, if we want
intakes = pd.read_csv('data/Austin_Animal_Center_Intakes_053122.csv')
# Check it out
intakes.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,06/29/2014 10:38:00 AM,June 2014,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [59]:
# Let's try merging on Animal ID
combined = outcomes.merge(intakes, 
                          how ='inner',
                          on='Animal ID')

In [60]:
# What was the result?
combined.head()

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [56]:
combined.shape

(180311, 24)

In [57]:
intakes.shape

(140055, 12)

In [58]:
outcomes.shape

(139957, 13)

Let's discuss/explore: did that work the way we expected?

- 


In [62]:
len(intakes.loc[intakes['Animal ID'] == 'A721033'])

33

In [63]:
len(intakes.loc[combined['Animal ID'] == 'A721033'])

1089

In [64]:
# We might want to try something different
# Can we clean something to make a better merge?
combined.drop_duplicates(subset='Animal ID', 
                         keep='last')

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
6,A720371,Moose,2016-02-15 00:00:00,Feb 2016,2015-10-08,Transfer,Partner,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
7,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,...,03/18/2014 07:11:00 AM,March 2014,12034 Research in Austin (TX),Stray,Nursing,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180306,A800114,*Cascade,2019-08-17 12:21:00,Aug 2019,2019-05-30,Adoption,NaN,Cat,Neutered Male,2 months,...,05/26/2022 11:45:00 AM,May 2022,14011 Fm 969 in Austin (TX),Stray,Normal,Cat,Neutered Male,2 years,Domestic Shorthair,Brown Tabby
180307,A858462,NaN,2022-05-30 15:26:00,May 2022,2020-05-30,Euthanasia,Rabies Risk,Other,Unknown,2 years,...,05/30/2022 01:26:00 PM,May 2022,800 W 5Th St in Austin (TX),Wildlife,Injured,Other,Unknown,2 years,Bat,Brown
180308,A858365,A858365,2022-05-30 15:08:00,May 2022,2020-05-28,Euthanasia,Rabies Risk,Other,Unknown,2 years,...,05/28/2022 03:39:00 PM,May 2022,4215 Far West Blvd in Austin (TX),Wildlife,Normal,Other,Unknown,2 years,Bat,Brown
180309,A858101,A858101,2022-05-29 12:50:00,May 2022,2021-11-25,Transfer,Partner,Cat,Intact Female,6 months,...,05/25/2022 01:00:00 PM,May 2022,4415 Lariena Drive in Austin (TX),Stray,Normal,Cat,Intact Female,5 months,Domestic Shorthair Mix,Black/White


In [65]:
# Try again
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180311 entries, 0 to 180310
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Animal ID               180311 non-null  object        
 1   Name_x                  138080 non-null  object        
 2   DateTime_x              180311 non-null  datetime64[ns]
 3   MonthYear_x             180311 non-null  object        
 4   Date of Birth           180311 non-null  datetime64[ns]
 5   Outcome Type            180277 non-null  object        
 6   Outcome Subtype         70394 non-null   object        
 7   Animal Type_x           180311 non-null  object        
 8   Sex upon Outcome        180310 non-null  object        
 9   Age upon Outcome        180290 non-null  object        
 10  Breed_x                 180311 non-null  object        
 11  Color_x                 180311 non-null  object        
 12  Calculated Age in Days  180311

# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [ ]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

## See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2